<a href="https://colab.research.google.com/github/upunaprosk/ADWISER/blob/master/seminar_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Word Embeddings

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
!pip install --upgrade nltk gensim bokeh

In [2]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2024-01-08 10:27:33--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2024-01-08 10:27:33--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc72f68f7dd4cd49a780d36962c3.dl.dropboxusercontent.com/cd/0/get/CK-wc0EkOUGD5QlZO0HsdKWzafIDk3JIL8xKmqFkXrgtyTdUNuoBNoHs8MGamX9UsBiJEJP2pRjzqCe_AntPOJmwdB7KFYGHFiVDgzKtHjFfZqC_yL99ltbBPyV3Kq5PzIA/file?dl=1# [following]
--2024-01-08 10:27:33--  https://uc72f68f7dd4cd49a780d36962c3.dl.dropboxusercontent.com/cd/0/get/CK-wc0EkOUGD5QlZO0HsdKWzafIDk3JIL8xKmqFkXrgtyTdUNuoBNoHs8MGamX9UsBiJEJP2pRjzqCe_AntPOJmwdB7KFYGHFiVDgzKtHjFfZqC_yL99ltbBPyV3Kq5PzI

In [3]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [4]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [5]:
# TASK: lowercase everything and extract tokens with tokenizer.
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(i.lower()) for i in data]

In [6]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [7]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings.

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [8]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok,
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [9]:
# now you can get word vectors !
model.get_vector('anything')

array([-3.7567108 ,  2.3874726 ,  1.6828706 ,  2.2734332 ,  0.46206796,
        1.4877135 ,  1.1555454 , -4.5726585 ,  0.54174596,  0.6694055 ,
       -0.8468227 ,  0.68929774,  4.8111324 ,  0.13175492,  1.2290444 ,
       -0.6592003 , -0.06520383, -0.31138787,  0.6584163 , -1.721026  ,
       -1.3044788 , -0.23699048, -3.398605  , -2.2932184 ,  2.9419968 ,
       -3.3556032 ,  0.68789536,  0.68609   ,  0.20279256, -1.1453501 ,
       -0.9166363 ,  2.2465355 ], dtype=float32)

In [10]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9723419547080994),
 ('butter', 0.9445321559906006),
 ('banana', 0.9288179874420166),
 ('vodka', 0.9276931285858154),
 ('sauce', 0.9270965456962585),
 ('chicken', 0.9259909987449646),
 ('fruit', 0.9233012199401855),
 ('cheese', 0.922906756401062),
 ('corn', 0.9208287596702576),
 ('honey', 0.909008800983429)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts.

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [11]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [12]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.4922019839286804),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [13]:
words = model.index_to_key[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [14]:
# for each word, compute it's vector with model
# word_vectors = # YOUR CODE
word_vectors = np.array([model.get_vector(word) for word in words])

In [15]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [16]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
# word_vectors_pca = # YOUR CODE

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
pca = PCA(n_components=2)
word_vectors_2d = pca.fit_transform(word_vectors)

# normalize vectors to make sure they have zero mean and unit variance
scaler = StandardScaler()
word_vectors_pca = scaler.fit_transform(word_vectors_2d)

In [17]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [18]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [19]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [20]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca

# Perform t-SNE to map the word vectors to a 2D plane
tsne = TSNE(n_components=2, random_state=0)  # You can adjust the perplexity and other parameters as needed
word_vectors_2d = tsne.fit_transform(word_vectors)

# Normalize the 2D word vectors to have zero mean and unit variance
scaler = StandardScaler()
word_tsne = scaler.fit_transform(word_vectors_2d)


In [21]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1053', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [22]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros


    # 1. Lowercase the phrase
    phrase = phrase.lower()

    # 2. Tokenize the phrase
    tokens = tokenizer.tokenize(phrase)

    # Initialize a vector to store the aggregated embeddings
    vector = np.zeros([model.vector_size], dtype='float32')

    # Initialize a counter to keep track of the number of valid words in the model's vocabulary
    valid_word_count = 0

    # Loop through each word in the tokenized phrase
    for token in tokens:
        # 3. Check if the word is in the model's vocabulary
        if model.key_to_index.get(token,0):
            # If the word is in the vocabulary, add its embedding to the vector
            vector += np.array(model.get_vector(token), dtype='float32')
            valid_word_count += 1

    # If there are valid words in the vocabulary, calculate the average embedding
    if valid_word_count > 0:
        vector /= valid_word_count

    return vector

In [23]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")
assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [24]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]
# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(i) for i in chosen_phrases])

In [25]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [26]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [27]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1102', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [28]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    # """
    # Compute the embedding vector for the query
    query_vector = get_phrase_embedding(query)

    # Compute cosine similarities between the query vector and all data vectors
    similarities = cosine_similarity([query_vector], data_vectors)[0]

    # Use argpartition to efficiently find the indices of the top-k most similar lines
    top_k_indices = np.argpartition(-similarities, k)[:k]

    # Sort the top-k indices by their similarity values
    top_k_indices = top_k_indices[np.argsort(-similarities[top_k_indices])]

    # Get the top-k most similar lines from the data
    top_k_lines = [data[i] for i in top_k_indices]
    return top_k_lines

    # return <YOUR CODE: top-k lines starting from most similar>

In [33]:
results = find_nearest(query="How do I enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [ ]:
find_nearest(query="<YOUR QUESTION>", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.